# Advanced Querying Mongo

Importing libraries and setting up connection

In [1]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient()

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [4]:
str_conn='mongodb://localhost:27017'

client = MongoClient(str_conn)

client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [5]:
client.list_database_names() 

db=client.companies

colec=db.companies

list(colec.find().limit(1))[0].keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

In [6]:
query = { 'name': 'Babelgum' } 
select = { 'name': 1, '_id': 0 }

list(colec.find(query, select))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [29]:
query= {'number_of_employees' : {'$gt':5000}}
filtro= {'name' : True, 'number_of_employees': True, '_id': 0}
        
list(colec.find(query, filtro).limit(3))

[{'name': 'Facebook', 'number_of_employees': 5299},
 {'name': 'eBay', 'number_of_employees': 15000},
 {'name': 'Cisco', 'number_of_employees': 63000}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [30]:
query = { '$and' : [{'founded_year': {'$lte': 2005}},
                     {'founded_year': {'$gte': 2000}}]}

filtro = {'name' : True, 'founded_year' : True, '_id':0}

list(colec.find(query, filtro).limit(3))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [31]:
query={'$and': [{'ipo.valuation_amount' : {'$gt':100000000}},
       {'founded_year' : {'$lt' : 2010}}]}

filtro = { 'name': True,'founded_year': True, 'ipo.valuation_amount': True, '_id':0}

list(colec.find(query, filtro).limit(3))

[{'name': 'Facebook',
  'founded_year': 2004,
  'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter',
  'founded_year': 2006,
  'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp',
  'founded_year': 2004,
  'ipo': {'valuation_amount': 1300000000}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [32]:
query={'$and': [{'number_of_employees' : {'$lt':1000}},
                {'founded_year' : {'$lt' : 2005}}]}
                
filtro= {'name' : True, 'number_of_employees': True, 'founded_year': True , '_id': 0}
        
list(colec.find(query, filtro).limit(3))

[{'name': 'AdventNet', 'number_of_employees': 600, 'founded_year': 1996},
 {'name': 'Digg', 'number_of_employees': 60, 'founded_year': 2004},
 {'name': 'Fox Interactive Media',
  'number_of_employees': 0,
  'founded_year': 1979}]

### 6. All the companies that don't include the `partners` field.

In [33]:
query = ({'partners' : { '$exists': 'False'}})

filtro= {'name' : True,'partners' :True, '_id': 0}

list(colec.find(query, filtro).limit(3))

[{'name': 'Wetpaint', 'partners': []},
 {'name': 'AdventNet', 'partners': []},
 {'name': 'Zoho', 'partners': []}]

### 7. All the companies that have a null type of value on the `category_code` field.

In [34]:
query = ({'category_code' : { '$exists': 'False'}})

filtro= {'name' : True,'category_code' :True, '_id': 0}

list(colec.find(query, filtro).limit(3))

[{'name': 'Wetpaint', 'category_code': 'web'},
 {'name': 'AdventNet', 'category_code': 'enterprise'},
 {'name': 'Zoho', 'category_code': 'software'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [35]:
query={'$and': [{'number_of_employees': 100},
                {'number_of_employees': {'$lt': 1000}}]}

filtro = {'name' :True, 'number_of_employees': True, '_id': 0}

list(colec.find(query, filtro).limit(3))

[{'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'hi5', 'number_of_employees': 100}]

### 9. Order all the companies by their IPO price in a descending order.

In [36]:
query = { 'ipo.valuation_amount':{ '$exists': True}}

filtro = { 'name':True, 'ipo.valuation_amount': True ,'_id': 0}

list(colec.find(query, filtro).sort('ipo.valuation_amount', -1).limit(5))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [37]:
query ={'$and': [{'number_of_employees': {'$lt': 50000}},
                 {'number_of_employees': {'$gt': 50}}]}

filtro = {'name': True, 'number_of_employees' : True, '_id' : 0}

list(colec.find(query, filtro).sort('number_of_employees', -1).limit(3))



[{'name': 'Telstra', 'number_of_employees': 39000},
 {'name': 'Bristol-Myers Squibb', 'number_of_employees': 35000},
 {'name': 'Nike', 'number_of_employees': 34300}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [38]:
query = {'$and' : [{ 'founded_month': {'$lte': 12}},
                   {'founded_month': {'$gt': 6}}]}

filtro = {'name' : 1, 'founded_month': True, '_id' : 0}

list(colec.find(query, filtro).limit(3))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [39]:
query = {'$and' : [{ 'founded_year': {'$lt': 2000}},
                   {'acquisition.price_amount': {'$gt': 1000000}}]}

filtro = {'name' : True, 'acquisition.price_amount': True, '_id' : 0}

list(colec.find(query, filtro).limit(3))       

[{'name': 'Postini', 'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep', 'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar', 'acquisition': {'price_amount': 25000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [40]:
query = {'acquisition.acquired_year': { '$gt' : 2010}}

filtro = {'name' : True, 'acquisition': True, '_id' : 0}
          
list(colec.find(query, filtro).sort('acquisition', -1).limit(3))

[{'name': 'T-Mobile',
  'acquisition': {'price_amount': 39000000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/',
   'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion',
   'acquired_year': 2011,
   'acquired_month': 3,
   'acquired_day': 20,
   'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}},
 {'name': 'Goodrich Corporation',
  'acquisition': {'price_amount': 18400000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html',
   'source_description': 'UTC shells out $18.4 billion for Goodrich',
   'acquired_year': 2011,
   'acquired_month': 9,
   'acquired_day': 22,
   'acquiring_company': {'name': 'United Technologies',
    'permalink': 'united-technologies'}}},
 {'name': 'LS

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [41]:
query = {'founded_year': { '$ne' : None}}

filtro = {'name' : True, 'founded_year' : True, '_id' : 0}

list(colec.find(query, filtro).sort('founded_year').limit(3))

[{'name': 'US Army', 'founded_year': 1800},
 {'name': 'SmallWorlds', 'founded_year': 1800},
 {'name': 'Alstrasoft', 'founded_year': 1800}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [42]:
query = { 'founded_day': {'$lte' : 7}}
                          
filtro= { 'name' : True, 'acquisition.price_amount': True, '_id':0}
                          
list(colec.find(query,filtro).sort('acquisition.price_amount', -1).limit(3))

[{'name': 'Netscape', 'acquisition': {'price_amount': 4200000000}},
 {'name': 'PayPal', 'acquisition': {'price_amount': 1500000000}},
 {'name': 'Zappos', 'acquisition': {'price_amount': 1200000000}}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [43]:
query = {'$and' : [{'category_code' : 'web'},
                   {'number_of_employees' : {'$gt' : 4000}}]}

filtro= { 'name' : True, 'number_of_employees': True, '_id':0}

list(colec.find(query, filtro).sort('number_of_employees', 1).limit(3))

[{'name': 'Expedia', 'number_of_employees': 4400},
 {'name': 'AOL', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'number_of_employees': 8657}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [44]:
query = {'$and' : [{'acquisition.price_amount' : {'$gt' : 1e7}},
                   {'acquisition.price_currency_code' : 'EUR'}]}

filtro = {'name' : True, 'acquisition.price_amount' : True, 'acquisition.price_currency_code' : True, '_id': 0}

list(colec.find(query, filtro).limit(3))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [46]:
query = {'acquisition.acquired_month' : {'$lte' : 3}}

filtro = {'name' :True, 'Adquisition': True, '_id': 0}

list(colec.find(query,filtro).limit(3))

[{'name': 'Kyte'}, {'name': 'NetRatings'}, {'name': 'blogTV'}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [23]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [24]:
# Your Code